In [ ]:
import os
import numpy as np
import pandas as pd

session = '691893_2023-10-05_12-46-57'
folderPath = 'F:/npOptoRecordings/withOpto/'
os.makedirs(os.path.join(folderPath, session, 'kilosort', 'postprocessed', 'experiment1_Record Node 104#Neuropix-PXI-100.ProbeA-AP_recording1', 'quality_metrics'), exist_ok=True)

wfDir = os.path.join(folderPath, session, 'sorted', 'allwaveforms')
timestampsDir = os.path.join(folderPath, session, 'sorted', 'timestamps')
laserTimesFile = os.path.join(timestampsDir, 'laserTimes.npy')
savePath = os.path.join(folderPath, session, 'figures')

if not os.path.exists(savePath):
    os.makedirs(savePath)

laserTimes = np.load(laserTimesFile)
allFiles = os.listdir(os.path.join(folderPath, session))
optoFile = next((s for s in allFiles if 'opto.csv' in s), None)
events = pd.read_csv(os.path.join(folderPath, session, optoFile))
metrics = pd.read_csv(os.path.join(folderPath, session, 'kilosort', 'postprocessed', 'experiment1_Record Node 104#Neuropix-PXI-100.ProbeA-AP_recording1', 'quality_metrics', 'metrics.csv'))
templates = np.load(os.path.join(wfDir, 'templates.npy'))

sites = events['site'].unique()
powers = events['power'].unique()
duration = events['duration'].unique()
intervals = events['pulse_interval'].unique()
pulseNum = events['num_pulses'].unique()
offset = duration + intervals
window = [0, 20/1000]
respWin = 20/1000
pThresh = 0.4
preLen = 500
postLen = 1000

allUnits = os.listdir(wfDir)
allUnits = [s for s in allUnits if 'allMeans.npy' in s]
IDnumber = [int(s[5:-13]) for s in allUnits]
IDnumber, allUnits = zip(*sorted(zip(IDnumber, allUnits)))

stepNum = 100
myMap = np.column_stack((np.linspace(1, 1, stepNum), np.linspace(1, 0, stepNum), np.linspace(1, 0, stepNum)))
isiV = np.zeros(len(allUnits))
isiThresh = 2/1000
maxChannel = []
for i in range(len(allUnits)):
    unitID = allUnits[i].split('_allMeans.npy')[0]
    currID = IDnumber[i]
    currTemplate = np.squeeze(templates[i,:,:])
    maxChannelTemp = np.argsort(np.min(currTemplate, axis=1))
    maxChannel.append(np.mean(maxChannelTemp[:3]))